In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
embeddings

In [ ]:
llm

In [ ]:
llm.invoke("What is the capital of France?").content

## Tools

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper, return_direct=True, verbose=True)

In [ ]:
print(tool.run("Who is Max Verstappen?"))

## Create your custom tool

In [ ]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Get the length of a word."""
    return len(word)


# We have to define a docstring for the tool to work properly, without it, it won't show up in the list of tools the agent can use and give error.

In [ ]:
get_word_length.invoke("Hello World")

In [ ]:
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

In [ ]:
add_numbers.invoke({"a": 5, "b": 10})

## Concept of Agents

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools

In [ ]:
tools = load_tools(["wikipedia"], llm=llm)

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


In [ ]:
agent.run("Hello how are you?")

In [ ]:
agent.run("Who is the current world champion in Formula 1 racing?")

In [ ]:
agent.run("What is GPT-2?")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily basically provides a search engine tool that can be used to fetch real-time information from the web.

In [ ]:
search = TavilySearchResults()

In [ ]:
search.run("Who is the current world champion in Formula 1 racing?")

In [ ]:
search.invoke("Who is the current world champion in Formula 1 racing?")

In [ ]:
tools = [search]

In [ ]:
from langchain import hub

In [ ]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
prompt.messages

In [ ]:
from langchain.agents import create_tool_calling_agent

agent1 = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=6,
    early_stopping_method="generate",   # <- if it’s stuck, ask LLM to write a final answer
)

In [ ]:
agent_executor.invoke({"input": "Who is John Wick?"})

## Self Ask with Search Agent Langgraph

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/John_Wick")
docs = loader.load()

In [ ]:
docs

In [ ]:
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [ ]:
vector  = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("Who is John Wick?")[0].page_content

In [ ]:
from langchain.tools.retriever import create_retriever_tool

In [ ]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="JohnWickRetriever",
    description="Useful for when you need to find information about John Wick from Wikipedia.",
)

In [ ]:
tools1 = [search, retriever_tool]

In [8]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [ ]:
agent2 = create_tool_calling_agent(llm, tools1, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent2, tools=tools1, verbose=True)

In [ ]:
agent_executor.invoke({"input": "What is Johns profession?"})

In [ ]:
agent_executor.invoke({"input": "What is the weather in New York?"})

In [ ]:
search2 = TavilySearchResults(k=3)
tool2 = [search2]

In [ ]:
agent3 = create_tool_calling_agent(llm, tool2, prompt)

In [ ]:
agent_executor1 = AgentExecutor(agent=agent3, tools=tool2, verbose=True, max_iterations=4, early_stopping_method="generate")

In [ ]:
agent_executor1.invoke({"input": "What is the weather in New York as of 29th August 2025?"})

## ReAct Agent (Reasoning and Action Agent)

In [9]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

In [ ]:
google_search = GoogleSerperAPIWrapper()

In [ ]:
tools = [
    Tool(
        name = "Web Search Functionality",
        func = google_search.run,
        description = "Useful for when you need to find information about current events or topics that are not in your training data. Input should be a search query.",
        verbose = True
    )
]

In [ ]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [ ]:
prompt = PromptTemplate.from_template(template)

In [ ]:
search_agent_react = create_react_agent(llm,tools,prompt)

In [ ]:
agent_executor = AgentExecutor(
    agent = search_agent_react,
    tools = tools,
    verbose = True,
    return_intermediate_steps=True,
    handle_parsing_errors=(
        "Please follow the required format. Either choose an Action and Action Input, "
        "or finish with 'Final Answer: <answer>'.")
)

In [ ]:
agent_executor.invoke({"input": "Who won the latest football world cup?"})

In [ ]:
agent_executor.invoke({"input": "Give me a list of movies in which spiderman is present?"})

## ReAct with Custom Tool

In [20]:
from langchain.tools import Tool
from langchain_core.tools import tool

In [21]:
@tool
def get_employee_id(name: str) -> str:
    """Return employee ID for the given employee name (case-insensitive)."""
    fake_employees = {
        "alice": "E001",
        "bob": "E002",
        "charlie": "E003",
        "diana": "E004",
        "evan": "E005",
        "fiona": "E006",
        "george": "E007",
        "hannah": "E008",
        "ian": "E009",
        "jasmine": "E010",
    }
    key = name.strip().lower()
    return fake_employees.get(key, "NOT_FOUND")

In [22]:
get_employee_id("Alice")

'E001'

In [24]:
# Custom tool for the Agent 
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")

In [25]:
tools = [get_employee_id, get_employee_salary]

In [26]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [27]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [28]:
agent = create_react_agent(llm,tools,prompt)

In [31]:
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True ,verbose=True)

In [32]:
agent_executor.invoke({"input": "What is the salary of Evan?"})



> Entering new AgentExecutor chain...


/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: The user is asking for the salary of Evan. To get the salary, I first need to find Evan's employee ID. I can use the `get_employee_id` tool for this. After I have the employee ID, I can use the `get_employee_salary` tool to find his salary.
Action: get_employee_id
Action Input: {"name": "Evan"}NOT_FOUND

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I could not find an employee with the name Evan. I cannot proceed to find the salary.
Final Answer: I am sorry, but I cannot find an employee with the name Evan.

> Finished chain.


{'input': 'What is the salary of Evan?',
 'output': 'I am sorry, but I cannot find an employee with the name Evan.'}